# 🚀 Tutorial de Streamlit: De un Análisis a una App Web Interactiva

**Objetivo:** Enseñar los fundamentos de Streamlit para construir una aplicación web (un *dashboard*) a partir de nuestros análisis de datos fotovoltaicos.

**Contexto:** Tomaremos nuestros datos simulados de 3 días y los gráficos de Seaborn/Matplotlib que ya creamos, y los pondremos en una página web interactiva donde un usuario pueda, por ejemplo, seleccionar qué día quiere ver.

---

## ⚠️ ¡Concepto Clave! Jupyter vs. Streamlit

* **Jupyter Notebook (`.ipynb`):** Es un entorno para *exploración* y análisis interactivo, celda por celda.
* **Streamlit (`.py`):** Es una biblioteca para *construir aplicaciones web*. Funciona ejecutando un script de Python completo (`.py`) desde la terminal.

### El Flujo de Trabajo de Streamlit

1.  **Escribir:** Creas un archivo (ej. `app.py`).
2.  **Ejecutar:** Abres tu terminal (Anaconda Prompt, PowerShell, Terminal, etc.) y ejecutas: 
    ```bash
    streamlit run app.py
    ```
3.  **Ver:** Streamlit abre automáticamente una pestaña en tu navegador que apunta a tu app (ej. `http://localhost:8501`).
4.  **Interactuar:** Cada vez que cambias un widget (como un menú desplegable) en la app, Streamlit **re-ejecuta el script `app.py` completo** de arriba a abajo.

## Paso 1: Instalación y Preparación de Datos

Primero, si no lo has hecho, instala Streamlit. Puedes ejecutar esto en una celda de código (añadiendo `!`) o en tu terminal.

In [2]:
# ¡Asegúrate de tener streamlit instalado!
# !pip install streamlit

# Importaciones que usaremos en nuestra app
import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración de Seaborn
sns.set_theme(style="darkgrid", palette="colorblind")

## Paso 2: Nuestra Función de Datos

Vamos a encapsular nuestra simulación de datos en una sola función. Esto es una **buena práctica**.

Usaremos el `df_long` del tutorial de Seaborn, ya que es perfecto para filtrar.

In [3]:
def simular_datos_pv():
    """Crea y devuelve el DataFrame de 72h en formato largo."""
    timestamps = pd.date_range(start='2025-07-01', periods=72, freq='h')
    horas_dia = np.arange(0, 72)
    ciclo_solar_base = np.sin(np.pi * (horas_dia % 24) / 24) ** 2
    irradiancia_base = np.maximum(0, ciclo_solar_base * 1000 + np.random.normal(0, 5, 72))

    # Días
    irradiancia_dia1 = irradiancia_base
    potencia_dia1 = irradiancia_dia1 * 0.15
    irradiancia_dia2 = np.maximum(0, irradiancia_base * 0.4 + np.random.normal(0, 20, 72))
    potencia_dia2 = irradiancia_dia2 * 0.15
    irradiancia_dia3 = irradiancia_base
    potencia_dia3 = irradiancia_dia3 * 0.15
    potencia_dia3[24+11] *= 0.3
    potencia_dia3[24+14] *= 0.4

    # Formato Ancho inicial
    df_wide = pd.DataFrame(
        data={
            'Potencia_Dia1': potencia_dia1, 'Irradiancia_Dia1': irradiancia_dia1,
            'Potencia_Dia2': potencia_dia2, 'Irradiancia_Dia2': irradiancia_dia2,
            'Potencia_Dia3': potencia_dia3, 'Irradiancia_Dia3': irradiancia_dia3
        },
        index=timestamps
    )

    # Transformar a Formato Largo
    df_reset = df_wide.reset_index().rename(columns={'index': 'Timestamp'})
    df_d1 = df_reset[['Timestamp', 'Irradiancia_Dia1', 'Potencia_Dia1']].rename(
        columns={'Irradiancia_Dia1': 'Irradiancia', 'Potencia_Dia1': 'Potencia'})
    df_d1['Dia'] = 'Día 1 (Soleado)'
    df_d2 = df_reset[['Timestamp', 'Irradiancia_Dia2', 'Potencia_Dia2']].rename(
        columns={'Irradiancia_Dia2': 'Irradiancia', 'Potencia_Dia2': 'Potencia'})
    df_d2['Dia'] = 'Día 2 (Nublado)'
    df_d3 = df_reset[['Timestamp', 'Irradiancia_Dia3', 'Potencia_Dia3']].rename(
        columns={'Irradiancia_Dia3': 'Irradiancia', 'Potencia_Dia3': 'Potencia'})
    df_d3['Dia'] = 'Día 3 (Parcial)'

    return pd.concat([df_d1, df_d2, df_d3])

# Probemos la función aquí en el notebook
df_app = simular_datos_pv()
print("Datos listos para la App:")
df_app.head()

Datos listos para la App:


,Timestamp,Irradiancia,Potencia,Dia
0,2025-07-01 00:00:00,4.734142,0.710121,Día 1 (Soleado)
1,2025-07-01 01:00:00,18.351222,2.752683,Día 1 (Soleado)
2,2025-07-01 02:00:00,64.247325,9.637099,Día 1 (Soleado)
3,2025-07-01 03:00:00,146.227801,21.934170,Día 1 (Soleado)
4,2025-07-01 04:00:00,239.016616,35.852492,Día 1 (Soleado)


## Lección 1: Creando nuestra primera App (`app_v1.py`)

Vamos a crear un script `app_v1.py` que:
1.  Importe `streamlit as st` y `pandas`.
2.  Tenga nuestra función de datos.
3.  Muestre un **título** (`st.title`).
4.  Muestre el **DataFrame** (`st.dataframe`).

**¡El truco!** `%%writefile app_v1.py` es un "comando mágico" de Jupyter que guarda el contenido de esta celda en un archivo llamado `app_v1.py` en la misma carpeta que este notebook.

**¡IMPORTANTE!** Después de ejecutar la celda de abajo, ve a tu terminal y ejecuta:
```bash
streamlit run app_v1.py
```

In [4]:
%%writefile app_v1.py
# Este es el contenido de nuestro PRIMER archivo .py

import streamlit as st
import pandas as pd
import numpy as np

# --- Función de Datos ---
# (Copiamos la función de la celda anterior)
def simular_datos_pv():
    timestamps = pd.date_range(start='2025-07-01', periods=72, freq='h')
    horas_dia = np.arange(0, 72)
    ciclo_solar_base = np.sin(np.pi * (horas_dia % 24) / 24) ** 2
    irradiancia_base = np.maximum(0, ciclo_solar_base * 1000 + np.random.normal(0, 5, 72))
    irradiancia_dia1 = irradiancia_base
    potencia_dia1 = irradiancia_dia1 * 0.15
    irradiancia_dia2 = np.maximum(0, irradiancia_base * 0.4 + np.random.normal(0, 20, 72))
    potencia_dia2 = irradiancia_dia2 * 0.15
    irradiancia_dia3 = irradiancia_base
    potencia_dia3 = irradiancia_dia3 * 0.15
    potencia_dia3[24+11] *= 0.3
    potencia_dia3[24+14] *= 0.4
    df_wide = pd.DataFrame(
        data={'Potencia_Dia1': potencia_dia1, 'Irradiancia_Dia1': irradiancia_dia1,
              'Potencia_Dia2': potencia_dia2, 'Irradiancia_Dia2': irradiancia_dia2,
              'Potencia_Dia3': potencia_dia3, 'Irradiancia_Dia3': irradiancia_dia3},
        index=timestamps)
    df_reset = df_wide.reset_index().rename(columns={'index': 'Timestamp'})
    df_d1 = df_reset[['Timestamp', 'Irradiancia_Dia1', 'Potencia_Dia1']].rename(
        columns={'Irradiancia_Dia1': 'Irradiancia', 'Potencia_Dia1': 'Potencia'})
    df_d1['Dia'] = 'Día 1 (Soleado)'
    df_d2 = df_reset[['Timestamp', 'Irradiancia_Dia2', 'Potencia_Dia2']].rename(
        columns={'Irradiancia_Dia2': 'Irradiancia', 'Potencia_Dia2': 'Potencia'})
    df_d2['Dia'] = 'Día 2 (Nublado)'
    df_d3 = df_reset[['Timestamp', 'Irradiancia_Dia3', 'Potencia_Dia3']].rename(
        columns={'Irradiancia_Dia3': 'Irradiancia', 'Potencia_Dia3': 'Potencia'})
    df_d3['Dia'] = 'Día 3 (Parcial)'
    return pd.concat([df_d1, df_d2, df_d3])

# --- Caching de Datos ---
# ¡Importante! Usamos un "decorador" de cache.
# Esto evita que Streamlit vuelva a simular los datos CADA VEZ 
# que el usuario hace clic en algo.
@st.cache_data
def cargar_datos():
    return simular_datos_pv()

# --- Construcción de la App ---
df = cargar_datos()

# st.title() -> Muestra un título grande
st.title('Mi Primer Dashboard de Energía Solar ☀️')

# st.write() -> Es el "print" de Streamlit, es mágico y muestra 
# (casi) cualquier cosa: texto, dataframes, markdown, etc.
st.write("Estos son los datos simulados de 3 días en formato largo.")

# st.dataframe() -> Muestra un DataFrame interactivo (puedes ordenar, etc.)
st.dataframe(df)

Writing app_v1.py


## Lección 2: Añadiendo Gráficos (`app_v2.py`)

¡Genial! Ahora vamos a añadir el gráfico de Seaborn que hicimos.

El comando clave es `st.pyplot(fig)`.

1.  Creamos una figura de Matplotlib/Seaborn (`fig, ax = ...`).
2.  Hacemos nuestro gráfico (`sns.lineplot(...)`).
3.  **NO usamos `plt.show()`**. En su lugar, le pasamos la figura a Streamlit: `st.pyplot(fig)`.

**Acción:** Detén la app anterior (Ctrl+C en la terminal) y ejecuta la nueva:
```bash
streamlit run app_v2.py
```

In [ ]:
%%writefile app_v2.py
# Versión 2: Añadiendo un gráfico

import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# --- Función de Datos (exactamente la misma) ---
@st.cache_data
def cargar_datos():
    timestamps = pd.date_range(start='2025-07-01', periods=72, freq='h')
    horas_dia = np.arange(0, 72)
    ciclo_solar_base = np.sin(np.pi * (horas_dia % 24) / 24) ** 2
    irradiancia_base = np.maximum(0, ciclo_solar_base * 1000 + np.random.normal(0, 5, 72))
    irradiancia_dia1 = irradiancia_base
    potencia_dia1 = irradiancia_dia1 * 0.15
    irradiancia_dia2 = np.maximum(0, irradiancia_base * 0.4 + np.random.normal(0, 20, 72))
    potencia_dia2 = irradiancia_dia2 * 0.15
    irradiancia_dia3 = irradiancia_base
    potencia_dia3 = irradiancia_dia3 * 0.15
    potencia_dia3[24+11] *= 0.3
    potencia_dia3[24+14] *= 0.4
    df_wide = pd.DataFrame(
        data={'Potencia_Dia1': potencia_dia1, 'Irradiancia_Dia1': irradiancia_dia1,
              'Potencia_Dia2': potencia_dia2, 'Irradiancia_Dia2': irradiancia_dia2,
              'Potencia_Dia3': potencia_dia3, 'Irradiancia_Dia3': irradiancia_dia3},
        index=timestamps)
    df_reset = df_wide.reset_index().rename(columns={'index': 'Timestamp'})
    df_d1 = df_reset[['Timestamp', 'Irradiancia_Dia1', 'Potencia_Dia1']].rename(
        columns={'Irradiancia_Dia1': 'Irradiancia', 'Potencia_Dia1': 'Potencia'})
    df_d1['Dia'] = 'Día 1 (Soleado)'
    df_d2 = df_reset[['Timestamp', 'Irradiancia_Dia2', 'Potencia_Dia2']].rename(
        columns={'Irradiancia_Dia2': 'Irradiancia', 'Potencia_Dia2': 'Potencia'})
    df_d2['Dia'] = 'Día 2 (Nublado)'
    df_d3 = df_reset[['Timestamp', 'Irradiancia_Dia3', 'Potencia_Dia3']].rename(
        columns={'Irradiancia_Dia3': 'Irradiancia', 'Potencia_Dia3': 'Potencia'})
    df_d3['Dia'] = 'Día 3 (Parcial)'
    return pd.concat([df_d1, df_d2, df_d3])

# --- Construcción de la App ---
sns.set_theme(style="darkgrid", palette="colorblind")
df = cargar_datos()

st.title('Dashboard de Energía Solar v2 📈')

# st.header() -> Un subtítulo
st.header('Producción Total de los 3 Días')

# --- Creación del Gráfico ---
# 1. Creamos la figura y los ejes (¡como en el notebook de Matplotlib!)
fig, ax = plt.subplots(figsize=(12, 5))

# 2. Usamos Seaborn (¡como en el notebook de Seaborn!)
sns.lineplot(data=df, x='Timestamp', y='Potencia', hue='Dia', ax=ax)
ax.set_title('Comparación de Producción PV')
ax.set_xlabel('Fecha y Hora')
ax.set_ylabel('Potencia (kW simulados)')

# 3. En lugar de plt.show(), usamos st.pyplot()
st.pyplot(fig)

# --- Mostrar los datos (opcional) ---
st.header('Datos Crudos')
st.dataframe(df)

## Lección 3: ¡Interactividad! (`app_v3.py`)

Esta es la **magia** de Streamlit. Vamos a añadir *widgets* (controles).

Usaremos `st.selectbox()` para permitir al usuario elegir un día. El valor que elija el usuario se guardará en una variable de Python.

Recuerda: ¡cada vez que el usuario cambie el `selectbox`, el script se re-ejecutará! El `@st.cache_data` que pusimos evitará que los datos se simulen cada vez.

**Acción:** Detén la app anterior (Ctrl+C) y ejecuta:
```bash
streamlit run app_v3.py
```

In [ ]:
%%writefile app_v3.py
# Versión 3: ¡Interactividad!

import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# --- Función de Datos (exactamente la misma) ---
@st.cache_data
def cargar_datos():
    timestamps = pd.date_range(start='2025-07-01', periods=72, freq='h')
    horas_dia = np.arange(0, 72)
    ciclo_solar_base = np.sin(np.pi * (horas_dia % 24) / 24) ** 2
    irradiancia_base = np.maximum(0, ciclo_solar_base * 1000 + np.random.normal(0, 5, 72))
    irradiancia_dia1 = irradiancia_base
    potencia_dia1 = irradiancia_dia1 * 0.15
    irradiancia_dia2 = np.maximum(0, irradiancia_base * 0.4 + np.random.normal(0, 20, 72))
    potencia_dia2 = irradiancia_dia2 * 0.15
    irradiancia_dia3 = irradiancia_base
    potencia_dia3 = irradiancia_dia3 * 0.15
    potencia_dia3[24+11] *= 0.3
    potencia_dia3[24+14] *= 0.4
    df_wide = pd.DataFrame(
        data={'Potencia_Dia1': potencia_dia1, 'Irradiancia_Dia1': irradiancia_dia1,
              'Potencia_Dia2': potencia_dia2, 'Irradiancia_Dia2': irradiancia_dia2,
              'Potencia_Dia3': potencia_dia3, 'Irradiancia_Dia3': irradiancia_dia3},
        index=timestamps)
    df_reset = df_wide.reset_index().rename(columns={'index': 'Timestamp'})
    df_d1 = df_reset[['Timestamp', 'Irradiancia_Dia1', 'Potencia_Dia1']].rename(
        columns={'Irradiancia_Dia1': 'Irradiancia', 'Potencia_Dia1': 'Potencia'})
    df_d1['Dia'] = 'Día 1 (Soleado)'
    df_d2 = df_reset[['Timestamp', 'Irradiancia_Dia2', 'Potencia_Dia2']].rename(
        columns={'Irradiancia_Dia2': 'Irradiancia', 'Potencia_Dia2': 'Potencia'})
    df_d2['Dia'] = 'Día 2 (Nublado)'
    df_d3 = df_reset[['Timestamp', 'Irradiancia_Dia3', 'Potencia_Dia3']].rename(
        columns={'Irradiancia_Dia3': 'Irradiancia', 'Potencia_Dia3': 'Potencia'})
    df_d3['Dia'] = 'Día 3 (Parcial)'
    return pd.concat([df_d1, df_d2, df_d3])

# --- Construcción de la App ---
sns.set_theme(style="darkgrid", palette="colorblind")
df = cargar_datos()

st.title('Dashboard Interactivo de Energía Solar ⚡')

# --- WIDGET: st.selectbox ---
st.header("Análisis por Día")
st.write("Usa el menú desplegable para filtrar los datos y gráficos.")

# 1. Obtenemos las opciones para el selectbox
# .unique() nos da los nombres de los días: ['Día 1', 'Día 2', 'Día 3']
opciones_dias = ['Todos los días'] + list(df['Dia'].unique())

# 2. Creamos el selectbox
dia_seleccionado = st.selectbox("Selecciona un período:", opciones=opciones_dias)

# --- Lógica de Filtrado ---
# 3. Filtramos el DataFrame basado en la selección
if dia_seleccionado == 'Todos los días':
    df_filtrado = df
    titulo_grafico = 'Producción Total (3 Días)'
else:
    df_filtrado = df[df['Dia'] == dia_seleccionado]
    titulo_grafico = f'Producción del {dia_seleccionado}'

# --- Gráfico (ahora es dinámico) ---
# 4. Creamos el gráfico usando el df_filtrado
fig, ax = plt.subplots(figsize=(12, 5))
sns.lineplot(data=df_filtrado, x='Timestamp', y='Potencia', hue='Dia', ax=ax)
ax.set_title(titulo_grafico)
ax.set_xlabel('Fecha y Hora')
ax.set_ylabel('Potencia (kW simulados)')
st.pyplot(fig)

# --- Datos (ahora son dinámicos) ---
# 5. Mostramos el dataframe filtrado
st.header(f"Datos Crudos para: {dia_seleccionado}")
st.dataframe(df_filtrado)


## Lección 4: Mejorando el Layout (Sidebar, Columnas y Métricas) (`app_final.py`)

Una app real necesita un buen layout.

* `st.sidebar`: Mueve los controles (widgets) a una barra lateral para limpiar la página principal.
* `st.columns()`: Organiza elementos uno al lado del otro.
* `st.metric()`: Muestra KPIs (Key Performance Indicators) como números grandes.

**Acción:** ¡Esta es la versión final! Detén la app anterior (Ctrl+C) y ejecuta:
```bash
streamlit run app_final.py
```

In [ ]:
%%writefile app_final.py
# Versión Final: Layout Profesional

import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# --- Configuración de la Página ---
# st.set_page_config() debe ser el PRIMER comando de Streamlit
st.set_page_config(layout="wide", page_title="Dashboard PV")

# --- Función de Datos (con cache) ---
@st.cache_data
def cargar_datos():
    timestamps = pd.date_range(start='2025-07-01', periods=72, freq='h')
    horas_dia = np.arange(0, 72)
    ciclo_solar_base = np.sin(np.pi * (horas_dia % 24) / 24) ** 2
    irradiancia_base = np.maximum(0, ciclo_solar_base * 1000 + np.random.normal(0, 5, 72))
    irradiancia_dia1 = irradiancia_base
    potencia_dia1 = irradiancia_dia1 * 0.15
    irradiancia_dia2 = np.maximum(0, irradiancia_base * 0.4 + np.random.normal(0, 20, 72))
    potencia_dia2 = irradiancia_dia2 * 0.15
    irradiancia_dia3 = irradiancia_base
    potencia_dia3 = irradiancia_dia3 * 0.15
    potencia_dia3[24+11] *= 0.3
    potencia_dia3[24+14] *= 0.4
    df_wide = pd.DataFrame(
        data={'Potencia_Dia1': potencia_dia1, 'Irradiancia_Dia1': irradiancia_dia1,
              'Potencia_Dia2': potencia_dia2, 'Irradiancia_Dia2': irradiancia_dia2,
              'Potencia_Dia3': potencia_dia3, 'Irradiancia_Dia3': irradiancia_dia3},
        index=timestamps)
    df_reset = df_wide.reset_index().rename(columns={'index': 'Timestamp'})
    df_d1 = df_reset[['Timestamp', 'Irradiancia_Dia1', 'Potencia_Dia1']].rename(
        columns={'Irradiancia_Dia1': 'Irradiancia', 'Potencia_Dia1': 'Potencia'})
    df_d1['Dia'] = 'Día 1 (Soleado)'
    df_d2 = df_reset[['Timestamp', 'Irradiancia_Dia2', 'Potencia_Dia2']].rename(
        columns={'Irradiancia_Dia2': 'Irradiancia', 'Potencia_Dia2': 'Potencia'})
    df_d2['Dia'] = 'Día 2 (Nublado)'
    df_d3 = df_reset[['Timestamp', 'Irradiancia_Dia3', 'Potencia_Dia3']].rename(
        columns={'Irradiancia_Dia3': 'Irradiancia', 'Potencia_Dia3': 'Potencia'})
    df_d3['Dia'] = 'Día 3 (Parcial)'
    return pd.concat([df_d1, df_d2, df_d3])

# --- Cargar Datos ---
sns.set_theme(style="darkgrid", palette="colorblind")
df = cargar_datos()

# --- BARRA LATERAL (SIDEBAR) ---
# Todo lo que esté dentro de 'with st.sidebar:' aparecerá a la izquierda
with st.sidebar:
    st.header("Filtros y Controles")
    opciones_dias = ['Todos los días'] + list(df['Dia'].unique())
    dia_seleccionado = st.selectbox("Selecciona un período:", opciones=opciones_dias)
    
    st.markdown("---") # Un separador
    
    # st.checkbox() -> Devuelve True si está marcado, False si no
    ver_datos = st.checkbox("Mostrar datos crudos", value=False)

# --- PÁGINA PRINCIPAL ---
st.title("Dashboard de Rendimiento Fotovoltaico ☀️")
st.markdown(f"### Análisis para: *{dia_seleccionado}*")

# --- Lógica de Filtrado ---
if dia_seleccionado == 'Todos los días':
    df_filtrado = df
else:
    df_filtrado = df[df['Dia'] == dia_seleccionado]

# --- KPIs (Métricas) en Columnas ---
st.header("Indicadores Clave (KPIs)")

# Calculamos los KPIs
potencia_max = df_filtrado['Potencia'].max()
energia_total = df_filtrado['Potencia'].sum() # Como los datos son horarios, la suma es kWh
irradiancia_media = df_filtrado[df_filtrado['Irradiancia'] > 0]['Irradiancia'].mean()

# st.columns(3) -> crea 3 columnas
col1, col2, col3 = st.columns(3)

# st.metric() -> Muestra un KPI
col1.metric(label="Potencia Pico", value=f"{potencia_max:.1f} kW")
col2.metric(label="Energía Total Generada", value=f"{energia_total:.0f} kWh")
col3.metric(label="Irradiancia Diurna Media", value=f"{irradiancia_media:.0f} W/m²")

st.markdown("---")

# --- Gráficos en Columnas ---
st.header("Análisis Gráfico")
col_graf1, col_graf2 = st.columns(2)

with col_graf1:
    # Gráfico 1: Serie de Tiempo de Potencia
    fig1, ax1 = plt.subplots(figsize=(8, 4))
    sns.lineplot(data=df_filtrado, x='Timestamp', y='Potencia', hue='Dia', ax=ax1, legend=False)
    ax1.set_title("Producción de Potencia")
    ax1.set_xlabel("Hora")
    ax1.set_ylabel("Potencia (kW)")
    st.pyplot(fig1)

with col_graf2:
    # Gráfico 2: Relación Irradiancia vs Potencia
    fig2, ax2 = plt.subplots(figsize=(8, 4))
    sns.scatterplot(data=df_filtrado, x='Irradiancia', y='Potencia', hue='Dia', ax=ax2, legend=False)
    ax2.set_title("Curva de Potencia (Irradiancia vs. Potencia)")
    ax2.set_xlabel("Irradiancia (W/m²)")
    ax2.set_ylabel("Potencia (kW)")
    st.pyplot(fig2)

# --- Mostrar Datos Crudos (Condicional) ---
if ver_datos:
    st.markdown("---")
    st.header(f"Datos Crudos para: {dia_seleccionado}")
    st.dataframe(df_filtrado)


## Conclusión y Desafío

¡Felicidades! Acabas de pasar de un script de análisis estático a una aplicación web interactiva y profesional.

Has aprendido a:

1.  Usar los comandos básicos: `st.title`, `st.header`, `st.write`.
2.  Mostrar datos con `st.dataframe`.
3.  Mostrar gráficos de Matplotlib/Seaborn con `st.pyplot(fig)`.
4.  Hacer la app interactiva con **widgets** como `st.selectbox` y `st.checkbox`.
5.  Organizar el layout con `st.sidebar` y `st.columns`.
6.  Mostrar KPIs con `st.metric`.
7.  Optimizar la app con `@st.cache_data`.

**Desafío:**

Intenta añadir un nuevo widget en la `sidebar`. Un `st.slider()` para filtrar por un rango de potencia. (Pista: `pot_min, pot_max = st.slider(...)` y luego filtra el `df_filtrado` aún más).